# Titanic Logistic Regression

Here I explore another algorithm for checking Titanic survival - Logistic Regression.

## Setup

In [2]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# read data
data = pd.read_csv("../data/titanic_train.csv")

## Ensure data does not contain empty values

In [3]:
data.isnull().sum() # making sure there is no empty values by counting

Passengerid    0
Age            0
Fare           0
Sex            0
sibsp          0
zero           0
zero.1         0
zero.2         0
zero.3         0
zero.4         0
zero.5         0
zero.6         0
Parch          0
zero.7         0
zero.8         0
zero.9         0
zero.10        0
zero.11        0
zero.12        0
zero.13        0
zero.14        0
Pclass         0
zero.15        0
zero.16        0
Embarked       2
zero.17        0
zero.18        0
survived       0
dtype: int64

Data contains nulls in Embarked - we fill it with 0 (we assume they didn't embark the ship):

In [4]:
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].min())
# data.head()

## Bagging

In [5]:
# segment data
x = data.drop("survived", axis=1)
y = data["survived"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)
# x_train.shape, y_test.shape #show training data amounts

## Model

In [6]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train) # we look for corespondencies

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

## Accuracy

In [7]:
accuracy = model.score(x_test, y_test)
accuracy

0.8435114503816794

## Predictions and probability for whole test set

Exploring of outcomes (I wanted to use knowledge I gained earlier).

In [8]:
prediction = model.predict(x_test)
proba = model.predict_proba(x_test)

print("Survival prediction for every person:\n", prediction, "\n and propability of that: ", proba)

Survival prediction for every person:
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1
 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1
 0 1 1] 
 and propability of that:  [[0.98840918 0.01159082]
 [0.93200483 0.06799517]
 [0.98104446 0.01895554]
 [0.94141085 0.05858915]
 [0.90367127 0.09632873]
 [0.38884929 0.61115071]
 [0.88154344 0.11845656]
 [0.40354407 0.59645593]
 [0.86358332 0.13641668]
 [0.7697844  0.2302156 ]
 [0.99159913 0.00840087]
 [0.9890106  0.0109894 ]
 [0.92777388 0.07222612]
 [0.51103932 0.48896068]
 [0.98193656 0.01806344]
 [0.69348504 0.30651496]


## Improvement - Confusion Matrix

We got the accuracy of around .84 which is great but accuracy might be lying. For example, model might predict everyone died anyway and get high accuracy.

Thus we need to go **deeper** and get **better**.

In [9]:
from sklearn.metrics import confusion_matrix # import

y_pred = model.predict(x_test) # predictions for all of testing people.

cm = confusion_matrix(y_test, y_pred)
cm # what model got right what wrong - [predicted survived person survived, predicted survived person didn't survive], [predicted didn't survive person survived, predicted didn't survive person didn't survive]

array([[177,  12],
       [ 29,  44]])

## Precision and Recall

In [10]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       189
           1       0.79      0.60      0.68        73

    accuracy                           0.84       262
   macro avg       0.82      0.77      0.79       262
weighted avg       0.84      0.84      0.84       262



What does it tell us?

Model can find death easily, but miss many survivors.

It means that our model predicts more deaths, because that behaviour is being rewarded. To make the model not skip the survivors we need to change one crucial aspect - `class_weight`. It will make the model value survival more.


## New model

In [11]:
model_balanced = LogisticRegression(max_iter=1000, class_weight="balanced")

model_balanced.fit(x_train, y_train)

y_prediction_balanced = model_balanced.predict(x_test)

print(confusion_matrix(y_test, y_prediction_balanced))
print(classification_report(y_test, y_prediction_balanced))

[[146  43]
 [ 14  59]]
              precision    recall  f1-score   support

           0       0.91      0.77      0.84       189
           1       0.58      0.81      0.67        73

    accuracy                           0.78       262
   macro avg       0.75      0.79      0.76       262
weighted avg       0.82      0.78      0.79       262

